![pipeline_of_aq_data_process](http://p3rz3gu1u.bkt.clouddn.com/2018-04-24-aq_data_process.png)
<caption><center> **Figure 1**: pipeline of aq data process</center></caption>

In [28]:
# Using pandas to process data
from collections import Counter
import numpy as np
import pandas as pd
import datetime
from matplotlib import pyplot as plt
%matplotlib inline
from scipy.stats import pearsonr

from utils.data_util import load_ld_aq_data, load_bj_aq_data

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1. 数据载入

In [169]:
stations_to_predict = ['BL0','CD1','CD9','GN0','GN3','GR4','GR9','HV1','KF1','LW2','MY7','ST5','TH4'] # 13个
other_stations = ['BX1', 'BX9', 'CR8', 'CT2', 'CT3', 'GB0', 'HR1', 'KC1', 'LH0', 'RB7', 'TD5']        # 11个
features = ['NO2 (ug/m3)', 'PM10 (ug/m3)', 'PM2.5 (ug/m3)']

all_features = []
for station in stations_to_predict :
    for feature in features:
        station_feature = station + "_" + feature
        all_features.append(station_feature)

In [170]:
ld_aq_data, stations, ld_aq_stations, ld_aq_stations_merged = load_ld_aq_data()

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [171]:
ld_aq_stations_merged.shape

(10921, 73)

一共 24 个站，每个站有3个特征，加上一个 order 列，因此共有 73 列。其中有13个站是需要预测的数据，11个站不需要，因此只使用13个站的数据，而另外11个站只用于做就近的数据补全

In [172]:
print("最早的日期：", ld_aq_stations_merged.index.min())
print("最晚的日期：", ld_aq_stations_merged.index.max())

最早的日期： 2017-01-01 00:00:00
最晚的日期： 2018-04-01 00:00:00


### 2. 重复日期的值除去

- 伦敦没有重复日期
- 虽然数据中存在着很多的缺失值，但是在一些时间点上数据是重复的，因此首先去除重复的时间点。
- 思路是：
    1. 首先让数字成为 df index
    2. 遍历 index ，找出对应的时间的重复值，删除后出现的重复时间

In [173]:
df_merged = ld_aq_stations_merged
df_merged["time"] = df_merged.index
df_merged.set_index("order", inplace=True)
print("重复值去除之前，共有数据数量", df_merged.shape[0])

used_times = []
for index in df_merged.index :
    time = df_merged.loc[index]["time"]
    if time not in used_times :
        used_times.append(time)
    else : 
        df_merged.drop([index], inplace=True)
        
print("重复值去除之后，共有数据数量", df_merged.shape[0])
df_merged.set_index("time", inplace=True)

重复值去除之前，共有数据数量 10921
重复值去除之后，共有数据数量 10921


### 3. 缺失值的分析

- 缺失值分析包括
    - 整小时的缺失：哪些时间节点上，所有站点的所有特征都缺失了
    - 某个小时的某个站点的所有数据缺失
    - 某个小时的某个站点的某个数据缺失
- 对于上述第一种情况，伦敦不存在所有站点的整小时缺失。
- 对于后两种情况，使用距离该站的有数据的最近一个站的数据，直接作为该站的数据。

In [174]:
min_time = df_merged.index.min()
max_time = df_merged.index.max()

min_time = datetime.datetime.strptime(min_time, '%Y-%m-%d %H:%M:%S')
max_time = datetime.datetime.strptime(max_time, '%Y-%m-%d %H:%M:%S')
delta_all = max_time - min_time
print("在空气质量数据时间段内，总共应该有 %d 个小时节点。" %(delta_all.total_seconds()/3600 + 1))
print("实际的时间节点数是 %d" %(df_merged.shape[0]))
print("伦敦没有整小时的数据缺失。")

在空气质量数据时间段内，总共应该有 10921 个小时节点。
实际的时间节点数是 10921
伦敦没有整小时的数据缺失。


#### 3.1 某个小时某个站点数据缺失
- 在没有全部缺失的小时处，某个站点会在某个小时出现全部数据缺失的情况。这种情况下，使用距离该站最近的站的数据对其进行补全。
- 或者某个站点的某个值缺失，此时使用相邻站点的数据补全

In [175]:
from utils.weather_data_util import get_station_locations, get_location_lists

In [176]:
aq_station_locations = pd.read_csv("./KDD_CUP_2018/London/location/London_AirQuality_Stations.csv")

In [178]:
aq_station_locations.head()

,Unnamed: 0,api_data,need_prediction,historical_data,Latitude,Longitude,SiteType,SiteName
0,BX9,True,NaN,True,51.465983,0.184877,Suburban,Bexley - Slade Green FDMS
1,BX1,True,NaN,True,51.465983,0.184877,Suburban,Bexley - Slade Green
2,BL0,True,True,True,51.522287,-0.125848,Urban Background,Camden - Bloomsbury
3,CD9,True,True,True,51.527707,-0.129053,Roadside,Camden - Euston Road
4,CD1,True,True,True,51.544219,-0.175284,Kerbside,Camden - Swiss Cottage


In [179]:
aq_station_locations = aq_station_locations[["Unnamed: 0", "Latitude", "Longitude"]]

In [180]:
aq_station_locations.rename(index=str, columns={"Unnamed: 0":"stationName", "Latitude":"latitude", "Longitude":"longitude"}, inplace=True)

In [181]:
aq_station_locations.head()

,stationName,latitude,longitude
0,BX9,51.465983,0.184877
1,BX1,51.465983,0.184877
2,BL0,51.522287,-0.125848
3,CD9,51.527707,-0.129053
4,CD1,51.544219,-0.175284


对于一个空气质量站点，将其他站点按照距该站点距离的大小关系排列，并保存成列表

In [182]:
for index_t in aq_station_locations.index:
    row_t = aq_station_locations.loc[index_t]
    # location of target station
    long_t = row_t["longitude"]
    lati_t = row_t["latitude"]
    # column name
    station_name = row_t["stationName"]
    
    # add a new column to df
    all_dis = []
    for index in aq_station_locations.index:
        row = aq_station_locations.loc[index]
        long = row['longitude']
        lati = row['latitude']
        dis = np.sqrt((long-long_t)**2 + (lati-lati_t)**2)
        all_dis.append(dis)
    
    aq_station_locations[station_name] = all_dis

In [183]:
# 不同站之间的距离关系
aq_station_locations

,stationName,latitude,longitude,BX9,BX1,BL0,CD9,CD1,CT2,CT3,...,HV1,LH0,KC1,KF1,LW2,RB7,TD5,ST5,TH4,MY7
0,BX9,51.465983,0.184877,0.000000,0.000000,0.315785,0.319941,0.368561,0.293436,0.266969,...,0.058542,0.626919,0.402157,0.402157,0.224697,0.145294,0.532047,0.335425,0.199425,0.344146
1,BX1,51.465983,0.184877,0.000000,0.000000,0.315785,0.319941,0.368561,0.293436,0.266969,...,0.058542,0.626919,0.402157,0.402157,0.224697,0.145294,0.532047,0.335425,0.199425,0.344146
2,BL0,51.522287,-0.125848,0.315785,0.315785,0.000000,0.006296,0.054083,0.022701,0.048817,...,0.331312,0.317552,0.087653,0.087653,0.098347,0.214024,0.240228,0.133937,0.117653,0.028743
3,CD9,51.527707,-0.129053,0.319941,0.319941,0.006296,0.000000,0.049091,0.027854,0.053127,...,0.334585,0.314988,0.084701,0.084701,0.103814,0.216039,0.239567,0.138993,0.121297,0.026054
4,CD1,51.544219,-0.175284,0.368561,0.368561,0.054083,0.049091,0.000000,0.076746,0.102138,...,0.381465,0.272052,0.044686,0.044686,0.152305,0.259425,0.207756,0.158538,0.169396,0.029970
5,CT2,51.514525,-0.104516,0.293436,0.293436,0.022701,0.027854,0.076746,0.000000,0.026759,...,0.310040,0.338093,0.109171,0.109171,0.075991,0.195315,0.257089,0.130631,0.096099,0.050712
6,CT3,51.513847,-0.077766,0.266969,0.266969,0.048817,0.053127,0.102138,0.026759,0.000000,...,0.283311,0.364724,0.135917,0.135917,0.054463,0.170033,0.282114,0.139993,0.069358,0.077315
7,CR8,51.410039,-0.127523,0.317370,0.317370,0.112260,0.117678,0.142427,0.106989,0.115117,...,0.350918,0.323823,0.140404,0.140404,0.109258,0.264015,0.218616,0.025111,0.158784,0.115711
8,GN0,51.490532,0.074003,0.113559,0.113559,0.202358,0.206431,0.255003,0.180124,0.153549,...,0.134894,0.515633,0.289110,0.289110,0.114707,0.079453,0.424658,0.238247,0.085990,0.230823
9,GR4,51.452580,0.070766,0.114896,0.114896,0.208605,0.213475,0.262561,0.185906,0.160671,...,0.150980,0.513670,0.292387,0.292387,0.112651,0.117533,0.417270,0.221656,0.100857,0.235966


In [184]:
# 以每一个站的名字为 key，以其他站的名字组成的列表为 value list，列表中从前向后距离越来越远
near_stations = {}
for index_t in aq_station_locations.index:
    target_station_name = aq_station_locations.loc[index_t]['stationName']
    ordered_stations_names = aq_station_locations.sort_values(by=target_station_name)['stationName'].values[1:]
    near_stations[target_station_name] = ordered_stations_names

In [185]:
# 举个例子：dingling_aq 附近的、按照距离排序的站的名字
near_stations['BX1']

array(['BX1', 'HV1', 'GN3', 'GB0', 'GN0', 'GR4', 'GR9', 'RB7', 'TH4',
       'LW2', 'CT3', 'CT2', 'BL0', 'CR8', 'CD9', 'ST5', 'MY7', 'CD1',
       'KC1', 'KF1', 'HR1', 'TD5', 'LH0'], dtype=object)

#### 3.3 个别缺失的处理

In [187]:
def get_estimated_value(station_name, feature_name, near_stations, row):
    '''
    为 feature 寻找合理的缺失值的替代。
    Args:
        near_stations : a dict of {station : near stations}
    '''   
    near_stations = near_stations[station_name]    # A list of nearest stations
    for station in near_stations :                 # 在最近的站中依次寻找非缺失值
        feature = station + "_" +feature_name
        if not pd.isnull(row[feature]):
            return row[feature]
        else :
            return 0

In [188]:
for index in df_merged.index :
    row = df_merged.loc[index]
    for feature in row.index :
        # print(feature)
        if pd.isnull(row[feature]) :
            
            elements = feature.split("_")                  # feature example： KC1_NO2 (ug/m3)
            station_name = elements[0]                     # KC1
            feature_name = elements[1]                   # NO2 (ug/m3)
            row[feature] = get_estimated_value(station_name, feature_name, near_stations, row)

In [189]:
# 现在数据中没有缺失值了 :)
print(pd.isnull(df_merged).any().any())

False


只利用 all_features 中的特征

In [194]:
df_merged = df_merged[all_features]

In [195]:
df_merged.to_csv("test/ld_aq_data.csv")

### 4. 数据归一化

In [196]:
describe = df_merged.describe()
describe.to_csv("test/ld_aq_describe.csv")

In [197]:
df_norm = (df_merged - describe.loc['mean']) / describe.loc['std']
df_norm.to_csv("test/ld_aq_norm_data.csv")

# ChangeLog
- 0424
    - 对原空气质量数据进行了处理，包括：
        - 删去重复日期
        - 缺失值分析
            - 整体缺失数据(指所有站点的所有特征在这一时刻均无数据)
                - 缺失不超过5小时：使用前后线性插值的方式
                    - 暂时没有考虑 “基于天的周期性”
                - 缺失超过5小时：放弃该时刻的值，使用 NAN 替代
            - 局部缺失数据(指某些站点的某些特征在这一刻没有数据)
                - 解决方法：收集距离该站最近的且该特征有数值的特征，并填充该值
    - 下载新数据，并进行数据融合
- 0425
    - 实现了对数据的正则化
    - 这在概念上是不是错误的？因为是在整个数据上对数据进行了统计，而正确的应该是仅在训练集上对数据进行统计